In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
def plot_to_notebook(time_sec, in_signal_n_samples,out_signal=None):
    plt.figure()
    plt.subplot(1,1,1)
    plt.xlabel('Time (usec)')
    plt.grid()
    plt.plot(time_sec[:n_samples]*1e6, in_signal[:n_samples], 'y-', label='input signal')
    
    if out_signal is not None:
          plt.plot(time_sec[:n_samples]*1e6, out_signal[:n_samples], 'g-',linewidth=2, label='FIR OUTPUT')

In [ ]:
import numpy as np
Total_time= 0.002
Samp_freq = 100e6
numb_samples = int(Total_time*Samp_freq)
t= np.linspace(0, Total_time, numb_samples endpoint=False)
samples = 1000*np.sin(0.2e6*2*np.pi*t) + 1500*np.cos(46e6*2*np.pi*t) + 2000*np.sin(12e6*2*np.pi*t)
samples_32bit = samples.astype(np.int32)
print('No. of samples: ',len(samples_32bit))
plot_to_notebook(t,samples_32bit,1000)

In [ ]:
from pynq import Overlay
import pynq.lib.dma
from pynq import Xlnk
import numpy as np

overlay = Overlay('/home/xilinx/pynq/overlays/fir_accelerator/fir_accelerator.bit')

dma = overlay.filter.fir_dma

xlnk = Xlnk()
in_buffer = xlnk.cma_array(shape=(n,), dtype=np.int32)
out_buffer = xlnk.cma_array(shape=(n,), dtype=np.int32)

np.copyto(in_buffer,samples)

import time

start_time = time.time()
dma.sendchannel.transfer(in_buffer)
dma.recvchannel.transfer(out_buffer)
dma.sendchannel.wait()
dma.recvchannel.wait()
stop_time = time.time()

hard_exec_t= stop_time - start_time
print('Hardware performance time: ', hard_exec_t)

plot_to_notebook(t,samples_32bit, 1000, out_signal = out_buffer)
in_buffer.close()
out_buffer.close()